Content
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

data = pd.read_csv('creditcard.csv')
pd.options.display.max_columns = 31
data.tail()

c:\users\tiani\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Time         V1         V2        V3        V4        V5  \
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9       V10       V11       V12  \
284802 -2.606837 -4.918215  7.305334  1.914428  4.356170 -1.593105  2.711941   
284803  1.058415  0.024330  0.294869  0.584800 -0.975926 -0.150189  0.915802   
284804  3.031260 -0.296827  0.708417  0.432454 -0.484782  0.411614  0.063119   
284805  0.623708 -0.686180  0.679145  0.392087 -0.399126 -1.933849 -0.962886   
284806 -0.649617  1.577006 -0.414650  0.486180 -0.915427 -1.040458 -0.031513   

             V13       V14       V15       V16       V17       V18       V19  \
284802 -0.689256  4.626942 -0.924459  1.107641  1.991691  0.510632 -0.682920   
284803  1.214756 -0.675143  1.164931 -0.711757 -0.025693 -1.221179 -1.545556   
284804 -0.183699 -0.510602  1.329284  0.140716  0.313502  0.395652 -0.577252   
284805 -1.042082  0.449624  1.962563 -0.608577  0.509928  1.113981  2.897849   
284806 -0.188093 -0.084316  0.041333 -0.302620 -0.660377  0.167430 -0.256117   

             V20       V21       V22       V23       V24       V25       V26  \
284802  1.475829  0.213454  0.111864  1.014480 -0.509348  1.436807  0.250034   
284803  0.059616  0.214205  0.924384  0.012463 -1.016226 -0.606624 -0.395255   
284804  0.001396  0.232045  0.578229 -0.037501  0.640134  0.265745 -0.087371   
284805  0.127434  0.265245  0.800049 -0.163298  0.123205 -0.569159  0.546668   
284806  0.382948  0.261057  0.643078  0.376777  0.008797 -0.473649 -0.818267   

             V27       V28  Amount  Class  
284802  0.943651  0.823731    0.77      0  
284803  0.068472 -0.053527   24.79      0  
284804  0.004455 -0.026561   67.88      0  
284805  0.108821  0.104533   10.00      0  
284806 -0.002415  0.013649  217.00      0

In [2]:
#Class of 1 means there's a fraud and 0 there's not fraud
numEachClass = data['Class'].value_counts()
numPositives = numEachClass.tolist()[1]
numNegatives = numEachClass.tolist()[0]
totalNumSamples = len(data)
print("Total number of data samples" + repr(totalNumSamples))
print('Positive samples: ' + repr(numPositives) + "\taccount for: " + repr(numPositives/totalNumSamples*100) + "%")
print("Negative samples: " + repr(numNegatives) + "\taccount for: " + repr(numNegatives/totalNumSamples*100) + "%")

Total number of data samples284807
Positive samples: 492	account for: 0.1727485630620034%
Negative samples: 284315	account for: 99.82725143693798%


In [3]:
# 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset.
# Let's clean it up to make it easier to understand
timeCol = data.Time
for time in timeCol.values:
    time /= 3600
    time = time % 24
print(timeCol)

0              0.0
1              0.0
2              1.0
3              1.0
4              2.0
5              2.0
6              4.0
7              7.0
8              7.0
9              9.0
10            10.0
11            10.0
12            10.0
13            11.0
14            12.0
15            12.0
16            12.0
17            13.0
18            14.0
19            15.0
20            16.0
21            17.0
22            18.0
23            18.0
24            22.0
25            22.0
26            23.0
27            23.0
28            23.0
29            23.0
            ...   
284777    172764.0
284778    172764.0
284779    172766.0
284780    172766.0
284781    172766.0
284782    172767.0
284783    172768.0
284784    172768.0
284785    172768.0
284786    172768.0
284787    172769.0
284788    172770.0
284789    172770.0
284790    172771.0
284791    172774.0
284792    172774.0
284793    172775.0
284794    172777.0
284795    172778.0
284796    172780.0
284797    172782.0
284798    17

In [4]:
#set variables
X = data.loc[:, ~data.columns.isin(['Time', 'Amount', 'Class'])]
Y = data['Class']
X

V1         V2        V3        V4        V5        V6  \
0       -1.359807  -0.072781  2.536347  1.378155 -0.338321  0.462388   
1        1.191857   0.266151  0.166480  0.448154  0.060018 -0.082361   
2       -1.358354  -1.340163  1.773209  0.379780 -0.503198  1.800499   
3       -0.966272  -0.185226  1.792993 -0.863291 -0.010309  1.247203   
4       -1.158233   0.877737  1.548718  0.403034 -0.407193  0.095921   
5       -0.425966   0.960523  1.141109 -0.168252  0.420987 -0.029728   
6        1.229658   0.141004  0.045371  1.202613  0.191881  0.272708   
7       -0.644269   1.417964  1.074380 -0.492199  0.948934  0.428118   
8       -0.894286   0.286157 -0.113192 -0.271526  2.669599  3.721818   
9       -0.338262   1.119593  1.044367 -0.222187  0.499361 -0.246761   
10       1.449044  -1.176339  0.913860 -1.375667 -1.971383 -0.629152   
11       0.384978   0.616109 -0.874300 -0.094019  2.924584  3.317027   
12       1.249999  -1.221637  0.383930 -1.234899 -1.485419 -0.753230   
13       1.069374   0.287722  0.828613  2.712520 -0.178398  0.337544   
14      -2.791855  -0.327771  1.641750  1.767473 -0.136588  0.807596   
15      -0.752417   0.345485  2.057323 -1.468643 -1.158394 -0.077850   
16       1.103215  -0.040296  1.267332  1.289091 -0.735997  0.288069   
17      -0.436905   0.918966  0.924591 -0.727219  0.915679 -0.127867   
18      -5.401258  -5.450148  1.186305  1.736239  3.049106 -1.763406   
19       1.492936  -1.029346  0.454795 -1.438026 -1.555434 -0.720961   
20       0.694885  -1.361819  1.029221  0.834159 -1.191209  1.309109   
21       0.962496   0.328461 -0.171479  2.109204  1.129566  1.696038   
22       1.166616   0.502120 -0.067300  2.261569  0.428804  0.089474   
23       0.247491   0.277666  1.185471 -0.092603 -1.314394 -0.150116   
24      -1.946525  -0.044901 -0.405570 -1.013057  2.941968  2.955053   
25      -2.074295  -0.121482  1.322021  0.410008  0.295198 -0.959537   
26       1.173285   0.353498  0.283905  1.133563 -0.172577 -0.916054   
27       1.322707  -0.174041  0.434555  0.576038 -0.836758 -0.831083   
28      -0.414289   0.905437  1.727453  1.473471  0.007443 -0.200331   
29       1.059387  -0.175319  1.266130  1.186110 -0.786002  0.578435   
...           ...        ...       ...       ...       ...       ...   
284777   2.079137  -0.028723 -1.343392  0.358000 -0.045791 -1.345452   
284778  -0.764523   0.588379 -0.907599 -0.418847  0.901528 -0.760802   
284779   1.975178  -0.616244 -2.628295 -0.406246  2.327804  3.664740   
284780  -1.727503   1.108356  2.219561  1.148583 -0.884199  0.793083   
284781  -1.139015  -0.155510  1.894478 -1.138957  1.451777  0.093598   
284782  -0.268061   2.540315 -1.400915  4.846661  0.639105  0.186479   
284783  -1.796092   1.929178 -2.828417 -1.689844  2.199572  3.123732   
284784  -0.669662   0.923769 -1.543167 -1.560729  2.833960  3.240843   
284785   0.032887   0.545338 -1.185844 -1.729828  2.932315  3.401529   
284786  -2.076175   2.142238 -2.522704 -1.888063  1.982785  3.732950   
284787  -1.029719  -1.110670 -0.636179 -0.840816  2.424360 -2.956733   
284788   2.007418  -0.280235 -0.208113  0.335261 -0.715798 -0.751373   
284789  -0.446951   1.302212 -0.168583  0.981577  0.578957 -0.605641   
284790  -0.515513   0.971950 -1.014580 -0.677037  0.912430 -0.316187   
284791  -0.863506   0.874701  0.420358 -0.530365  0.356561 -1.046238   
284792  -0.724123   1.485216 -1.132218 -0.607190  0.709499 -0.482638   
284793   1.971002  -0.699067 -1.697541 -0.617643  1.718797  3.911336   
284794  -1.266580  -0.400461  0.956221 -0.723919  1.531993 -1.788600   
284795 -12.516732  10.187818 -8.476671 -2.510473 -4.586669 -1.394465   
284796   1.884849  -0.143540 -0.999943  1.506772 -0.035300 -0.613638   
284797  -0.241923   0.712247  0.399806 -0.463406  0.244531 -1.343668   
284798   0.219529   0.881246 -0.635891  0.960928 -0.152971 -1.014307   
284799  -1.775135  -0.004235  1.189786  0.331096  1.196063  5.519980   
284800   2.039560  -0.175233 -1.196825  0.234580 -0.008

In [5]:
# Make the categorical variable 'Class' into dummies.
#X = pd.concat([X, pd.get_dummies(data['Class'])], axis=1)

# Create training and test sets.
offset = int(X.shape[0] * 0.8)

# Put 80% of the data in the training set.
X_train, Y_train = X[:offset], Y[:offset]

# And put 20% in the test set.
X_test, Y_test = X[offset:], Y[offset:]

Y_train.sum()

417

In [7]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X, Y)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(X)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (Y != y_pred).sum()
))

Number of mislabeled points out of a total 284807 points : 241


In [8]:
from sklearn import ensemble

# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 5,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, Y_train)

predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

# Accuracy tables.
table_train = pd.crosstab(Y_train, predict_train, margins=True)
table_test = pd.crosstab(Y_test, predict_test, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All', 1]
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All', 0]

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All', 1]
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All', 0]

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(train_tI_errors, train_tII_errors, test_tI_errors, test_tII_errors))

Training set accuracy:
Percent Type I errors: 0.2872340425531915
Percent Type II errors: 0.0012430981696148152

Test set accuracy:
Percent Type I errors: 0.1935483870967742
Percent Type II errors: 0.0008782561346191003


In [9]:
table_test

col_0      0   1    All
Class                  
0      56881   6  56887
1         50  25     75
All    56931  31  56962

In [ ]:
# import sv classifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc = SVC()
cross_val_score(svc, X_n, y, cv=5)

In [ ]:
# Iterate through decision trees, each time using the residuals
# from the previous tree as the inputs.
from sklearn import tree
uniquex = data['Class'].unique()

# Store values from loops.
preds = pd.DataFrame()
labels = []

for m in range(0, 101):

    # Initialize and fit the tree. Set the max depth to 3.
    decision_tree = tree.DecisionTreeRegressor(max_depth=3)
    decision_tree.fit(X, Y)

    # Get and store predicted values.
    pred = decision_tree.predict(X)
    preds['pred{}'.format(m)] = pred

    # Residuals.
    Y = Y - pred

    # Output every 20 iterations.
    if m % 20 == 0:
        print('Weak learner {} R^2: {}'.format(m, decision_tree.score(X, Y)))
        labels = labels + [m]
        bestpred = preds.sum(axis=1)
        plt.plot(uniquex, np.poly1d(np.polyfit(
            data['Class'], bestpred, 1))(uniquex))

plt.legend(labels)
plt.xlabel('Original')
plt.ylabel('Predicted')
plt.show()
predicted = preds.sum(axis=1)

print('Overall R^2:')
print(np.corrcoef(data['Class'], predicted)[0, 1])

Weak learner 0 R^2: -1.9257145569218406
